In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.layers import Input, Conv2D, Dense, Activation, Flatten, BatchNormalization, MaxPooling2D, concatenate, Dropout, Lambda, RepeatVector, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.models import Model, Sequential
import numpy as np
import os
import tensorflow as tf1
import cv2
import imutils
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score, roc_curve, f1_score
import matplotlib.pyplot as plt

In [ ]:
!unzip /content/drive/MyDrive/DATASET.zip

Streaming output truncated to the last 5000 lines.
  inflating: DATASET/TRAIN/VECTORS/LYMPHOCYTE/_7_7998.txt  
  inflating: DATASET/TRAIN/VECTORS/LYMPHOCYTE/_7_8270.txt  
  inflating: DATASET/TRAIN/VECTORS/LYMPHOCYTE/_7_833.txt  
  inflating: DATASET/TRAIN/VECTORS/LYMPHOCYTE/_7_8489.txt  
  inflating: DATASET/TRAIN/VECTORS/LYMPHOCYTE/_7_8496.txt  
  inflating: DATASET/TRAIN/VECTORS/LYMPHOCYTE/_7_8508.txt  
  inflating: DATASET/TRAIN/VECTORS/LYMPHOCYTE/_7_8819.txt  
  inflating: DATASET/TRAIN/VECTORS/LYMPHOCYTE/_7_8937.txt  
  inflating: DATASET/TRAIN/VECTORS/LYMPHOCYTE/_7_8997.txt  
  inflating: DATASET/TRAIN/VECTORS/LYMPHOCYTE/_7_9001.txt  
  inflating: DATASET/TRAIN/VECTORS/LYMPHOCYTE/_7_9042.txt  
  inflating: DATASET/TRAIN/VECTORS/LYMPHOCYTE/_7_9226.txt  
  inflating: DATASET/TRAIN/VECTORS/LYMPHOCYTE/_7_9627.txt  
  inflating: DATASET/TRAIN/VECTORS/LYMPHOCYTE/_7_9767.txt  
  inflating: DATASET/TRAIN/VECTORS/LYMPHOCYTE/_7_9823.txt  
  inflating: DATASET/TRAIN/VECTORS/LYMPHOCYTE/_8_1

In [ ]:
image_size=(96,96)
code={"EOSINOPHIL":0,"LYMPHOCYTE":1,"MONOCYTE":2,"NEUTROPHIL":3}

images = []
vectors = []
labels = []

def findedges(image):
    gray = cv2.GaussianBlur(image, (1, 1), 0)
    edged = cv2.Canny(gray, 100, 400)
    edged = cv2.dilate(edged, None, iterations=1)
    edged = cv2.erode(edged, None, iterations=1)
    return edged

def getimageconturs(edged):
    contours = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    contours = sorted(contours, key=lambda x: cv2.contourArea(x))
    return contours

def getboxes(contours,orig):
    boxes = []
    centers = []
    for contour in contours:
        box = cv2.minAreaRect(contour)
        box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
        box = np.array(box, dtype="int")
        (tl, tr, br, bl) = box
        if (np.linalg.norm(tl - bl)) > 0 and (np.linalg.norm(tl - tr)) > 0:
            boxes.append(box)
    return boxes

def getcode(n):
    if type(n)==str:
        for x,y in code.items():
            if n==x:
                return y
    else:
        for x,y in code.items():
            if n==y:
                return x

In [ ]:
def load_and_preprocess_data(images_path, path):

    for label, folder_path in enumerate(path):
        vec_files = os.listdir(folder_path)
        for vec_file in vec_files:
            filename = vec_file.split(".")[0]
            class_name = getcode(label)
            image_path = f"{images_path}/{class_name}/{filename}.jpeg"
            vector_path = f"{folder_path}{vec_file}"

            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # add padding to the image to better detect cell at the edge
            image = cv2.copyMakeBorder(image,10,10,10,10,cv2.BORDER_CONSTANT,value=[198, 203, 208])

            #thresholding the image to get the target cell
            image1 = cv2.inRange(image,(80, 80, 180),(180, 170, 245))

            # openning errosion then dilation
            kernel = np.ones((3, 3), np.uint8)
            kernel1 = np.ones((5, 5), np.uint8)
            img_erosion = cv2.erode(image1, kernel, iterations=2)
            image1 = cv2.dilate(img_erosion, kernel1, iterations=5)

            #detecting the blood cell
            edgedImage = findedges(image1)
            edgedContours = getimageconturs(edgedImage)
            edgedBoxes =  getboxes(edgedContours, image.copy())

            if len(edgedBoxes)==0:
                continue
            # get the large box and get its cordinate
            last = edgedBoxes[-1]
            max_x = int(max(last[:,0]))
            min_x = int( min(last[:,0]))
            max_y = int(max(last[:,1]))
            min_y = int(min(last[:,1]))

            # draw the contour and fill it
            mask = np.zeros_like(image)
            cv2.drawContours(mask, edgedContours, len(edgedContours)-1, (255,255,255), -1)

            # any pixel but the pixels inside the contour is zero
            image[mask==0] = 0

            # extract th blood cell
            image = image[min_y:max_y, min_x:max_x]

            if (np.size(image)==0):
                continue
            # resize th image

            image = cv2.resize(image, image_size)
            image = image / 255.0  # Normalize pixel values to [0, 1]

            vector = np.loadtxt(vector_path, delimiter=',')

            images.append(image)
            vectors.append(vector)
            labels.append(label)


In [ ]:
paths = [
    '/content/DATASET/TRAIN/VECTORS/EOSINOPHIL/',
    '/content/DATASET/TRAIN/VECTORS/LYMPHOCYTE/',
    '/content/DATASET/TRAIN/VECTORS/MONOCYTE/',
    '/content/DATASET/TRAIN/VECTORS/NEUTROPHIL/',
]

test_paths = [
    '/content/DATASET/TEST/VECTORS/EOSINOPHIL/',
    '/content/DATASET/TEST/VECTORS/LYMPHOCYTE/',
    '/content/DATASET/TEST/VECTORS/MONOCYTE/',
    '/content/DATASET/TEST/VECTORS/NEUTROPHIL/'
]


In [ ]:
load_and_preprocess_data("/content/DATASET/TRAIN/IMAGES", paths)

In [ ]:
load_and_preprocess_data("/content/DATASET/TEST/IMAGES", test_paths)

In [ ]:

images,vectors,labels=shuffle(images,vectors,labels,random_state=10)

In [ ]:
images = np.array(images)
vectors = np.array(vectors)
labels = np.array(labels)

In [ ]:
# Generate Train and Test using 80/20 split
train_image, test_image, train_vector, test_vector, train_label, test_label = train_test_split(images, vectors, labels, test_size=0.2, random_state=42)

In [ ]:
train_image = np.array(train_image, dtype = 'float32')
train_vector = np.array(train_vector, dtype = 'float32')
train_label = np.array(train_label, dtype = 'int32')

test_image = np.array(test_image, dtype = 'float32')
test_vector = np.array(test_vector, dtype = 'float32')
test_label = np.array(test_label, dtype = 'int32')

In [ ]:
print(train_image.shape, train_vector.shape, train_label.shape)
print(test_image.shape, test_vector.shape, test_label.shape)

(9716, 96, 96, 3) (9716, 10) (9716,)
(2430, 96, 96, 3) (2430, 10) (2430,)


In [ ]:
def create_model_without_knowledge_vgg16():
    # Image input for VGG16 (resize images to 224x224)
    image_input = Input(shape=(96, 96, 3))
    base_model = VGG19(weights='imagenet', include_top=False, input_tensor=image_input)

    # Additional processing layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.25)(x)

    # Output layer
    output = Dense(4, activation='softmax')(x)  # Assuming 4 classes

    # Define the model
    model = Model(inputs=image_input, outputs=output)

    # Compile the model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [ ]:
def create_model_with_knowledge_vgg16():
    # Image input for VGG16 (resize images to 224x224)
    image_input = Input(shape=(96, 96, 3))
    base_model = VGG19(weights='imagenet', include_top=False, input_tensor=image_input)

    # Additional processing layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # Vector input
    vector_input = Input(shape=(10,))
    y = Dense(64)(vector_input)
    y = Activation('relu')(y)
    y = Dropout(0.25)(y)

    # Concatenate image and vector features
    z = concatenate([y, x])
    z = Dense(64)(z)
    z = Activation('relu')(z)
    z = Dropout(0.25)(z)

    z = Dense(64)(z)
    z = Activation('relu')(z)
    z = Dropout(0.25)(z)

    # Output layer
    output = Dense(4, activation='softmax')(z)  # Assuming 4 classes

    # Define the model
    model = Model(inputs=[image_input, vector_input], outputs=output)

    # Compile the model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [ ]:
# Create models
model_with_knowledge = create_model_with_knowledge_vgg16()
model_without_knowledge = create_model_without_knowledge_vgg16()

# Display model summaries
model_with_knowledge.summary()
model_without_knowledge.summary()

80134624/80134624 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 96, 96, 3)]          0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 96, 96, 64)           1792      ['input_1[0][0]']             
                                                                                                  
 block1_conv2 (Conv2D)       (None, 96, 96, 64)           36928     ['block1_conv1[0][0]']        
                                                                                                  
 block1_pool (MaxPooling2D)  (None, 48, 48, 64)           0         ['block1_conv2[0][0]']        
                             

In [ ]:
model_without_knowledge.fit(train_image, train_label, validation_split=0.1, epochs=30, batch_size=32)

Epoch 1/30
274/274 [==============================] - 47s 120ms/step - loss: 1.5493 - accuracy: 0.3106 - val_loss: 1.3173 - val_accuracy: 0.2840
Epoch 2/30
274/274 [==============================] - 29s 106ms/step - loss: 1.0862 - accuracy: 0.4649 - val_loss: 0.8892 - val_accuracy: 0.5113
Epoch 3/30
274/274 [==============================] - 30s 108ms/step - loss: 0.7761 - accuracy: 0.6256 - val_loss: 0.5390 - val_accuracy: 0.7953
Epoch 4/30
274/274 [==============================] - 29s 108ms/step - loss: 0.5161 - accuracy: 0.7756 - val_loss: 0.4058 - val_accuracy: 0.8189
Epoch 5/30
274/274 [==============================] - 30s 108ms/step - loss: 0.4133 - accuracy: 0.8339 - val_loss: 0.2912 - val_accuracy: 0.8807
Epoch 6/30
274/274 [==============================] - 30s 109ms/step - loss: 0.4051 - accuracy: 0.8445 - val_loss: 0.3779 - val_accuracy: 0.8374
Epoch 7/30
274/274 [==============================] - 30s 109ms/step - loss: 0.3284 - accuracy: 0.8752 - val_loss: 0.3294 - val_ac

In [ ]:
loss1,accuracy1=model_without_knowledge.evaluate(test_image, test_label)
print("the accuracy of vgg16 model (pretrained model) without knowledge is : ",accuracy1)

76/76 [==============================] - 4s 53ms/step - loss: 0.2571 - accuracy: 0.9012
the accuracy of vgg16 model (pretrained model) without knowledge is :  0.9012345671653748


In [ ]:
model_without_knowledge.save('/content/drive/My Drive/VGG16PretrainedModelWithoutKnowledge.keras')

In [ ]:
model_without_knowledge.save('/content/drive/My Drive/VGG16PretrainedModelWithoutKnowledge.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model_with_knowledge.fit([train_image, train_vector], train_label, validation_split=0.1, epochs=30, batch_size=32)

Epoch 1/30
274/274 [==============================] - 39s 118ms/step - loss: 0.4234 - accuracy: 0.8813 - val_loss: 0.0394 - val_accuracy: 0.9928
Epoch 2/30
274/274 [==============================] - 31s 114ms/step - loss: 0.0668 - accuracy: 0.9895 - val_loss: 0.0382 - val_accuracy: 0.9938
Epoch 3/30
274/274 [==============================] - 31s 113ms/step - loss: 0.0955 - accuracy: 0.9855 - val_loss: 0.0427 - val_accuracy: 0.9938
Epoch 4/30
274/274 [==============================] - 31s 115ms/step - loss: 0.0582 - accuracy: 0.9904 - val_loss: 0.0429 - val_accuracy: 0.9928
Epoch 5/30
274/274 [==============================] - 32s 115ms/step - loss: 0.0559 - accuracy: 0.9907 - val_loss: 0.0419 - val_accuracy: 0.9928
Epoch 6/30
274/274 [==============================] - 31s 112ms/step - loss: 0.0533 - accuracy: 0.9907 - val_loss: 0.0432 - val_accuracy: 0.9928
Epoch 7/30
274/274 [==============================] - 31s 112ms/step - loss: 0.0508 - accuracy: 0.9907 - val_loss: 0.0431 - val_ac

In [ ]:
loss2,accuracy2=model_with_knowledge.evaluate([test_image, test_vector], test_label)
print("the accuracy of VGG16 model (pretrained model) with knowledge is : ",accuracy2)

76/76 [==============================] - 3s 34ms/step - loss: 0.0408 - accuracy: 0.9926
the accuracy of VGG16 model (pretrained model) with knowledge is :  0.9925925731658936


In [ ]:
model_with_knowledge.save('/content/drive/My Drive/VGG16PretrainedModelWithKnowledge.keras')

In [ ]:
model_with_knowledge.save('/content/drive/My Drive/VGG16PretrainedModelWithKnowledge.h5')